In [18]:
import pandas as pd
from typing import Dict, List

class SimpleTSVValidator:
    def __init__(self):
        self.test_cases = []
        
    def load_tsv_file(self, file_path: str):
        """TSV 파일을 간단하게 로드"""
        try:
            # TSV로 읽기 (탭 구분자 사용)
            df = pd.read_csv(file_path, sep='\t', encoding='utf-8', header=1)
            
            # 컬럼명 정리 (끝에 붙은 쉼표 제거)
            df.columns = df.columns.str.rstrip(',').str.strip()
            
            # 데이터 정리 (각 셀의 끝에 붙은 쉼표 제거)
            for col in df.columns:
                if df[col].dtype == 'object':
                    df[col] = df[col].astype(str).str.rstrip(',').str.strip()
            
            print(f"✅ TSV 파일 로드 성공: {len(df)}개 행")
            print(f"📋 컬럼: {list(df.columns)}")
            
            # 테스트 케이스 생성
            for idx, row in df.iterrows():
                # NaN 값을 안전하게 처리하는 함수
                def safe_str(value):
                    if pd.isna(value) or str(value).lower() == 'nan':
                        return ''
                    return str(value).strip()
                
                # keys-정답을 리스트로 변환
                expected_keys_str = safe_str(row['keys-정답'])
                expected_keys = [k.strip() for k in expected_keys_str.split(',') if k.strip() and k.strip().lower() != 'nan']
                
                # keys_combined를 리스트로 변환
                actual_keys_str = safe_str(row['keys_combined'])
                actual_keys = [k.strip() for k in actual_keys_str.split(',') if k.strip() and k.strip().lower() != 'nan']
                
                # actual_keys에서 click_type 제외 (keys-정답에는 click_type이 없을 예정)
                actual_keys_filtered = [k for k in actual_keys if k != 'click_type']
                
                # values_combined를 리스트로 변환
                values_str = safe_str(row['values_combined'])
                values = [v.strip() for v in values_str.split(',') if v.strip() and v.strip().lower() != 'nan']
                
                test_case = {
                    'unique_id': idx + 1,
                    '기능': safe_str(row['기능']),
                    '경로': safe_str(row['경로']),
                    '활동': safe_str(row['활동']),
                    'page_id': safe_str(row['page_id']),
                    'act_type': safe_str(row['act_type']),
                    'click_type': safe_str(row['click_type']),
                    'expected_keys': expected_keys,  # keys-정답 (click_type 없음)
                    'actual_keys': actual_keys_filtered,  # keys_combined에서 click_type 제외
                    'actual_keys_original': actual_keys,  # 원본 actual_keys (참고용)
                    'values': values
                }
                self.test_cases.append(test_case)
                
            print(f"✅ 테스트 케이스 생성: {len(self.test_cases)}개")
            print(f"ℹ️  검증 시 actual_keys에서 click_type 제외됨")
            
        except Exception as e:
            print(f"❌ 오류: {e}")
            raise
    
    def validate_and_export(self, output_file: str):
        """검증 수행 및 결과 출력 (click_type 제외)"""
        results = []
        
        # NaN 값 안전 처리 함수
        def safe_str_result(value):
            if pd.isna(value) or str(value).lower() == 'nan':
                return ''
            return str(value).strip()
        
        for test_case in self.test_cases:
            unique_id = test_case['unique_id']
            actual_keys = test_case['actual_keys']  # click_type 제외된 키들
            expected_keys = test_case['expected_keys']
            values = test_case['values']
            actual_keys_original = test_case['actual_keys_original']  # 원본 키들
            
            # keys_combined가 비어있거나 actual_keys가 비어있으면 로그 누락
            if len(actual_keys) == 0 and len(expected_keys) > 0:
                results.append({
                    '고유번호': unique_id,
                    '기능': safe_str_result(test_case['기능']),
                    '경로': safe_str_result(test_case['경로']),
                    '활동': safe_str_result(test_case['활동']),
                    'page_id': safe_str_result(test_case['page_id']),
                    'act_type': safe_str_result(test_case['act_type']),
                    'click_type': safe_str_result(test_case['click_type']),
                    'key': 'LOG_MISSING',
                    'value': '로그 누락',
                    'pass': 'FAIL'
                })
                continue
            
            # expected_keys도 비어있고 actual_keys도 비어있으면 건너뛰기
            if len(actual_keys) == 0 and len(expected_keys) == 0:
                continue
            
            # key-value 매핑 (원본 키들 기준으로 매핑)
            key_value_map = {}
            for i, key in enumerate(actual_keys_original):
                if i < len(values):
                    key_value_map[key] = safe_str_result(values[i])
                else:
                    key_value_map[key] = ''
            
            actual_keys_set = set(actual_keys)  # click_type 제외된 키들
            expected_keys_set = set(expected_keys)
            
            # 예상 키 검증 (click_type 제외된 actual_keys와 비교)
            for key in expected_keys:
                value = key_value_map.get(key, 'MISSING')
                pass_status = 'PASS' if key in actual_keys_set else 'FAIL'
                
                results.append({
                    '고유번호': unique_id,
                    '기능': safe_str_result(test_case['기능']),
                    '경로': safe_str_result(test_case['경로']),
                    '활동': safe_str_result(test_case['활동']),
                    'page_id': safe_str_result(test_case['page_id']),
                    'act_type': safe_str_result(test_case['act_type']),
                    'click_type': safe_str_result(test_case['click_type']),
                    'key': key,
                    'value': safe_str_result(value),
                    'pass': pass_status
                })
            
            # 예상치 못한 키 확인 (click_type 제외된 actual_keys 기준)
            unexpected_keys = actual_keys_set - expected_keys_set
            for key in unexpected_keys:
                value = key_value_map.get(key, '')
                results.append({
                    '고유번호': unique_id,
                    '기능': safe_str_result(test_case['기능']),
                    '경로': safe_str_result(test_case['경로']),
                    '활동': safe_str_result(test_case['활동']),
                    'page_id': safe_str_result(test_case['page_id']),
                    'act_type': safe_str_result(test_case['act_type']),
                    'click_type': safe_str_result(test_case['click_type']),
                    'key': key,
                    'value': safe_str_result(value),
                    'pass': 'UNEXPECTED'
                })
        
        # 결과를 DataFrame으로 변환
        df_results = pd.DataFrame(results)
        
        # Excel로 저장
        df_results.to_excel(output_file, index=False, engine='openpyxl')
        
        # 요약 출력
        self._print_summary(results)
        print(f"✅ 결과 저장: {output_file}")
        
        return results

    def _print_summary(self, results: List[Dict]):
        """결과 요약"""
        df = pd.DataFrame(results)
        
        total = len(df)
        passed = len(df[df['pass'] == 'PASS'])
        failed = len(df[df['pass'] == 'FAIL'])
        unexpected = len(df[df['pass'] == 'UNEXPECTED'])
        log_missing = len(df[df['key'] == 'LOG_MISSING'])
        
        print(f"\n📊 검증 결과 (click_type 제외):")
        print(f"   전체: {total}개")
        print(f"   통과: {passed}개 ({passed/total*100:.1f}%)" if total > 0 else "   통과: 0개")
        print(f"   실패: {failed}개")
        print(f"   예상외: {unexpected}개")
        print(f"   로그누락: {log_missing}개")

# 사용 예시
def run_simple_validator():
    validator = SimpleTSVValidator()
    
    try:
        # TSV 파일 로드
        validator.load_tsv_file("tester.tsv")  # 또는 실제 파일 경로
        
        # 검증 수행
        validator.validate_and_export("qa_result_click_type_excluded.xlsx")
        
    except Exception as e:
        print(f"❌ 실행 오류: {e}")

if __name__ == "__main__":
    run_simple_validator()

✅ TSV 파일 로드 성공: 45개 행
📋 컬럼: ['기능', '경로', '활동', 'no', 'page_id', 'click_type', 'act_type', 'click_text', 'keys_combined', 'values_combined', 'key_count', 'keys-정답']
✅ 테스트 케이스 생성: 45개
ℹ️  검증 시 actual_keys에서 click_type 제외됨

📊 검증 결과 (click_type 제외):
   전체: 329개
   통과: 131개 (39.8%)
   실패: 106개
   예상외: 92개
   로그누락: 11개
✅ 결과 저장: qa_result_click_type_excluded.xlsx


In [13]:
df = pd.read_csv("tester.tsv", sep='\t', encoding='utf-8', header=1)
df

,기능,경로,활동,no,page_id,click_type,act_type,click_text,keys_combined,values_combined,key_count,keys-정답
0,라운드메인,메인,메인 퀵버튼 캐시전환 클릭,1.0,life-dev/main,퀵버튼,click,캐시전환 썸네일,"channel, page_url, click_text, el_order, os_name","Rround, https://life-dev.hectoinnovation.co.kr...",5.0,"el_order, click_text"
1,라운드메인,메인/캐시전환팝업,지금 전환 클릭,2.0,life-dev/main,NaN,click,지금 전환하기,"channel, page_url, click_text, os_name","Rround, https://life-dev.hectoinnovation.co.kr...",4.0,"channel, page_url, click_text, os_name"
2,라운드메인,메인,메인 퀵버튼 만보기 클릭,6.0,life-dev/main,퀵버튼,click,만보기 썸네일,"channel, page_url, click_text, el_order, os_name","Rround, https://life-dev.hectoinnovation.co.kr...",5.0,"el_order, click_text"
3,라운드메인,메인,메인 퀵버튼 로또 클릭,9.0,life-dev/main,퀵버튼,click,라운드로또 썸네일,"channel, page_url, click_text, el_order, os_name","Rround, https://life-dev.hectoinnovation.co.kr...",5.0,"el_order, click_text"
4,라운드메인,메인,메인 퀵버튼 휘슬 클릭,29.0,life-dev/main,퀵버튼,click,휘슬 썸네일,"channel, page_url, click_text, el_order, os_name","Rround, https://life-dev.hectoinnovation.co.kr...",5.0,"el_order, click_text"
5,라운드메인,메인_상단,메인 상단 카드 배너 클릭,12.0,life-dev/main,메인 상단 카드 배너,click,NaN,"channel, page_url, banner_position, el_order, ...","Rround, https://life-dev.hectoinnovation.co.kr...",5.0,"banner_id, banner_url, banner_text, el_order"
6,라운드메인,메인_상단,검색 창 클릭,14.0,life-dev/main,검색 창,click,NaN,"channel, page_url, os_name","Rround, https://life-dev.hectoinnovation.co.kr...",3.0,"channel, page_url, os_name"
7,라운드메인,메인_상단,검색 아이콘 클릭,16.0,life-dev/main,검색 아이콘,click,NaN,"channel, page_url, os_name","Rround, https://life-dev.hectoinnovation.co.kr...",3.0,"channel, page_url, os_name"
8,라운드메인,메인_모듈-누구나원하는상품,상품 클릭,2.0,life-dev/main,상품,click,쿠쿠 인스퓨어 25년형 초슬림 창문형에어컨 벽걸이형에어컨,"channel, page_url, click_text, prd_order, prd_...","Rround, https://life-dev.hectoinnovation.co.kr...",9.0,"module_id, module_name, module_order, el_order..."
9,라운드메인,메인_모듈-누구나원하는상품,상품 찜하기 클릭,5.0,life-dev/main,상품 찜하기,click,NaN,"channel, page_url, prd_code, prd_name, prd_pri...","Rround, https://life-dev.hectoinnovation.co.kr...",7.0,"module_id, module_name, module_order, el_order..."
